# 1.4 Подробнее о методах кластеризации. Часть 2

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.
Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

﻿id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...﻿
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

﻿id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...﻿

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

﻿38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...﻿

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

﻿33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)﻿

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.



In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [2]:
fn= open("checkins.cvs","w+")
with open('checkins.dat', 'r') as file_obj:
    for line in file_obj:
        s = line.replace(' ','')
        fn.write(s)

fn.close()         

In [3]:
df = pd.read_csv('checkins.cvs', header=0, sep='|',encoding="ascii")

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.dropna(axis=0, how='any', subset=[u'latitude', u'longitude'])

In [5]:
df = df.reset_index(drop=True)


In [6]:
X = df.loc[0:100000, [u'latitude',u'longitude']]

In [8]:
ms = MeanShift(bandwidth=0.1)
labels = ms.fit_predict(X)

In [9]:
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)


In [10]:
office_coords = [[33.751277, -118.188740],[25.867736, -80.324116], [51.503016, -0.075479], [52.378894, 4.885084], [39.366487, 117.036146],[-33.868457, 151.205134]]


In [11]:
from geopy.distance import geodesic
def find_nearest_office(sample, coords):
    nearest = 0
    best_distance = np.finfo(np.float128)
    for i in xrange(len(coords)):
        cur_distance =  geodesic(sample, coords[i]).miles
        if cur_distance<best_distance:
            best_distance = cur_distance 
        
    return best_distance 

In [12]:
centers = {}


In [18]:
for i in xrange(len(labels_unique)):
    n =  np.where(labels==i)
    #если число элементов кластера меньше 15, то пропускаем его
    if len(n[0]) <= 15:
        continue
    #получаем расстояние от текущего центра кластера до ближайшего офиса компании 
    centers[i] = find_nearest_office(ms.cluster_centers_[i],office_coords)
       


In [14]:
sorted(centers.items(),key=lambda x:x[1])

[(370, 0.511664343451444),
 (420, 0.5398192612027276),
 (419, 1.5533279019313413),
 (58, 2.159790544052751),
 (51, 4.643651562879575),
 (29, 8.692677591694473),
 (167, 11.462413168547034),
 (87, 12.405938191899171),
 (92, 12.479297048864403),
 (42, 13.031824594085165),
 (291, 15.297283492950612),
 (27, 17.715084103590986),
 (320, 18.674044088397018),
 (119, 19.126680225230164),
 (32, 19.730980565651418),
 (55, 20.230766873050133),
 (11, 21.582425263299726),
 (159, 23.363854357480996),
 (17, 24.302470995778588),
 (50, 26.088401514913745),
 (64, 26.31588769632372),
 (47, 26.51659971631632),
 (24, 27.56584285235236),
 (35, 28.256203674348225),
 (100, 30.91398300501197),
 (144, 36.52888459150488),
 (75, 36.61812011170779),
 (284, 37.07261626172531),
 (90, 37.11315461081331),
 (224, 37.45057147319702),
 (398, 39.77060736840808),
 (293, 45.57702588386442),
 (233, 46.382736205975),
 (247, 47.64294447770499),
 (143, 48.67324370801663),
 (497, 49.3256497398751),
 (366, 53.5114052269924),
 (226,

In [ ]:
ms.cluster_centers_[370]

In [ ]:
print find_nearest_office(ms.cluster_centers_[0],office_coords)

In [ ]:
print ms.cluster_centers_[0]
print len(coords)

In [ ]:
cur_distance =  geodesic(ms.cluster_centers_[0], office_coords[0]).miles

In [ ]:
print cur_distance

In [ ]:
print len(ms.cluster_centers_)
len(labels_unique)